# Deep contextualized word representations
## Matthew E. Peters, Mark Neumann, Mohit Iyyer, Matt Gardner, Christopher Clark, Kenton Lee and Luke Zettlemoyer
### Code
- https://github.com/blackbbc/NER
- https://github.com/sarveshsparab/DeepElmoEmbedNer



### Installing the required dependencies

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install tqdm
!pip install gensim

### Adding paths to sys paths

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../model')

### Supressing warning level messages in output 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Instantiating an object of the NER parent class implementation 

In [ ]:
from DeepElmoEmbedNer import DeepElmoEmbedNer

deen = DeepElmoEmbedNer()

### Instantiating the input files for the model
### 3 files required
- train : For the model to train
- dev : For the model to validate the training
- test : To evaluate the performance to the model

In [ ]:
file_dict = dict()
file_dict['train'] = '../data/sample/ner_test_input.txt'
file_dict['test'] = '../data/sample/ner_test_input.txt'
file_dict['dev'] = '../data/sample/ner_test_input.txt'

###  

In [ ]:
data = deen.read_dataset(file_dict, "CoNLL2003", None)

In [ ]:
groundTruth = deen.convert_ground_truth(data, None)

In [ ]:
model, sess, saver = deen.train(data, None, maxEpoch=1)

In [ ]:
predictions = deen.predict('../data/sample/ner_test_input.txt', None, writeInputToFile=False, model=model, sess=sess, saver=saver, trainedData=data['train'])

In [ ]:
deen.evaluate([col[3] for col in predictions], [col[3] for col in groundTruth], None)